In [1]:
!pip install sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 12.9 MB/s eta 0:00:00


In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
seed = 0
np.random.seed(seed)

from keras.layers import Dense, LeakyReLU, SimpleRNN, LSTM
from keras.models import Sequential
from keras.optimizers import Adam
from keras import models
from keras import layers

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import datetime as dt
import string
import re

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory


import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

import csv
import requests
from io import StringIO

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


#Memuat Dataset

In [4]:
app_reviews_df = pd.read_csv('dataset.csv')

In [5]:
jumlah_ulasan, jumlah_kolom = app_reviews_df.shape

In [6]:
app_reviews_df.head()

,Review
0,"Rusak scan,Qris sulit terbaca kadang tidak bis..."
1,"Apk udah bagus tapi masih berasa berat , kalau..."
2,Aplikasinya sangat bagus diawal2 nya terutama ...
3,Animasi promo yang kalian buat membuat aplikas...
4,aplikasi makin kesini makin lemot saja... pada...


# Menghapus Data Kosong dan Data Duplikat

In [7]:
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126000 entries, 0 to 125999
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Review  126000 non-null  object
dtypes: object(1)
memory usage: 984.5+ KB


In [8]:
clean_df = app_reviews_df.dropna()

In [10]:
clean_df.drop_duplicates(inplace=True)

In [11]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 124209 entries, 0 to 125999
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Review  124209 non-null  object
dtypes: object(1)
memory usage: 1.9+ MB


In [12]:
jumlah_ulasan_bersih, jumlah_kolom_bersih = clean_df.shape

#Preprocessing Text

In [13]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'#[A-Za-z0-9]+', '', text)
    text = re.sub(r'RT[\s]', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'[^\w\s]', '', text)

    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip(' ')

    return text

In [14]:
def casefoldingText(text):
    text = text.lower()
    return text

In [15]:
def tokenizingText(text):
    text = word_tokenize(text)
    return text

In [16]:
def filteringText(text):
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku','di','ga','ya','gaa','loh','kah','woi','woii','woy'])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

In [17]:
def stemmingText(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    words = text.split()

    stemmed_words = [stemmer.stem(word) for word in words]

    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

In [18]:
def toSentence(list_words):
    sentence = ' '.join(word for word in list_words)
    return sentence

In [19]:
slangwords = {'@': 'di', 'abis': 'habis', 'wtb': 'beli', 'masi': 'masih', 'wts': 'jual', 'wtt': 'tukar', 'bgt': 'banget', 'maks': 'maksimal'}

def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [22]:
clean_df['text_clean'] = clean_df['Review'].apply(cleaningText)
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)
clean_df['text_stopwords'] = clean_df['text_tokenizingText'].apply(filteringText)
clean_df['final_text'] = clean_df['text_stopwords'].apply(toSentence)

In [23]:
clean_df.head()

,Review,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopwords,final_text
0,"Rusak scan,Qris sulit terbaca kadang tidak bis...",Rusak scanQris sulit terbaca kadang tidak bisa...,rusak scanqris sulit terbaca kadang tidak bisa...,rusak scanqris sulit terbaca kadang tidak bisa...,"[rusak, scanqris, sulit, terbaca, kadang, tida...","[rusak, scanqris, sulit, terbaca, kadang, ngeb...",rusak scanqris sulit terbaca kadang ngebaca su...
1,"Apk udah bagus tapi masih berasa berat , kalau...",Apk udah bagus tapi masih berasa berat kalau ...,apk udah bagus tapi masih berasa berat kalau ...,apk udah bagus tapi masih berasa berat kalau b...,"[apk, udah, bagus, tapi, masih, berasa, berat,...","[apk, udah, bagus, berasa, berat, latar, gamba...",apk udah bagus berasa berat latar gambar apk g...
2,Aplikasinya sangat bagus diawal2 nya terutama ...,Aplikasinya sangat bagus diawal nya terutama t...,aplikasinya sangat bagus diawal nya terutama t...,aplikasinya sangat bagus diawal nya terutama t...,"[aplikasinya, sangat, bagus, diawal, nya, teru...","[aplikasinya, bagus, diawal, transaksinya, mem...",aplikasinya bagus diawal transaksinya membantu...
3,Animasi promo yang kalian buat membuat aplikas...,Animasi promo yang kalian buat membuat aplikas...,animasi promo yang kalian buat membuat aplikas...,animasi promo yang kalian buat membuat aplikas...,"[animasi, promo, yang, kalian, buat, membuat, ...","[animasi, promo, aplikasi, patah, patah, mengs...",animasi promo aplikasi patah patah mengscroll ...
4,aplikasi makin kesini makin lemot saja... pada...,aplikasi makin kesini makin lemot saja padahal...,aplikasi makin kesini makin lemot saja padahal...,aplikasi makin kesini makin lemot saja padahal...,"[aplikasi, makin, kesini, makin, lemot, saja, ...","[aplikasi, kesini, lemot, koneksi, bagus, siny...",aplikasi kesini lemot koneksi bagus sinyal man...


# Memberi Label pada Data

In [24]:
lexicon_positive = dict()
lexicon_negative = dict()

In [25]:
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')

In [26]:
if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')

    for row in reader:
        lexicon_positive[row[0]] = int(row[1])

else:
    print('Gagal')

In [27]:
if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')

    for row in reader:
        lexicon_negative[row[0]] = int(row[1])

else:
    print('Gagal')

In [28]:
def sentiment_analysis_lexicon_indonesia(text):
    score = 0

    for word in text:
        if(word in lexicon_positive):
            score = score + lexicon_positive[word]

    for word in text:
        if(word in lexicon_negative):
            score = score + lexicon_negative[word]

    polarity=''

    if(score>0):
        polarity = 'positive'
    elif(score<0):
        polarity = 'negative'
    else:
        polarity = 'neutral'

    return score, polarity

In [29]:
results = clean_df['text_stopwords'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]
print(clean_df['polarity'].value_counts())

polarity
negative    117321
neutral       6888
Name: count, dtype: int64


#Data Splitting dan Ektraksi Fitur

In [114]:
x = clean_df['final_text']
y = clean_df['polarity']

In [115]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

In [116]:
x_train, x_temp, y_train, y_temp = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

In [117]:
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8)

In [118]:
x_train_tfidf = tfidf.fit_transform(x_train)
x_val_tfidf = tfidf.transform(x_val)

In [119]:
features_df = pd.DataFrame(x_tfidf.toarray(), columns=tfidf.get_feature_names_out())

In [120]:
features_df

,admin,aja,aktif,akun,alasan,aman,aneh,apapun,apk,aplikasi,...,ulang,update,upgrade,verifikasi,versi,via,voucher,wa,wajah,yg
0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000
1,0.35928,0.114468,0.0,0.0,0.0,0.0,0.0,0.0,0.375633,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000
2,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.255763,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000
3,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.292220,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000
4,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.164382,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124204,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.619957,0.000000,0.0,0.000000
124205,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.291318,0.000000,0.0,0.000000
124206,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.173566,0.0,0.213559
124207,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000


#Pembuatan Model

In [121]:
model = Sequential([
    LSTM(8, return_sequences=True, input_shape=(1, x_train_tfidf.shape[1])),
    LSTM(16, return_sequences=True),
    LSTM(32),
    Dense(16),
    LeakyReLU(),
    Dense(3, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [122]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=1e-3), metrics=['accuracy'])

# Pelatihan Model

In [123]:
x_train_reshaped = np.reshape(x_train_tfidf.toarray(), (x_train_tfidf.shape[0], 1, x_train_tfidf.shape[1]))
model.fit(x_train_reshaped, y_train, epochs=10)

Epoch 1/10
3106/3106 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - accuracy: 0.9479 - loss: 0.2050
Epoch 2/10
3106/3106 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.9591 - loss: 0.0711
Epoch 3/10
3106/3106 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - accuracy: 0.9590 - loss: 0.0702
Epoch 4/10
3106/3106 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - accuracy: 0.9586 - loss: 0.0709
Epoch 5/10
3106/3106 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.9601 - loss: 0.0691
Epoch 6/10
3106/3106 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - accuracy: 0.9601 - loss: 0.0686
Epoch 7/10
3106/3106 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.9605 - loss: 0.0687
Epoch 8/10
3106/3106 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.9607 - loss: 0.0682
Epoch 9/10
3106/3106 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.9612 - loss: 0.0686
Epoch 10/10
3106/3106 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - accuracy: 0.9610 - loss: 0.0680


#Prediksi Model

In [127]:
x_val_reshaped = np.reshape(x_val_tfidf.toarray(), (x_val_tfidf.shape[0], 1, x_val_tfidf.shape[1]))

In [128]:
y_pred = model.predict(x_val_reshaped)

389/389 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


#Evaluasi Model

In [129]:
y_pred_classes = np.argmax(y_pred, axis=1)

In [130]:
accuracy = round(accuracy_score(y_pred_classes, y_val), 4)

In [131]:
print('Akurasi Model LSTM :', accuracy)

Akurasi Model LSTM : 0.9556


In [132]:
print(classification_report(y_val, y_pred_classes))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98     11693
           1       0.61      0.67      0.64       728

    accuracy                           0.96     12421
   macro avg       0.80      0.82      0.81     12421
weighted avg       0.96      0.96      0.96     12421



In [133]:
confusion_matrix(y_val, y_pred_classes)

array([[11385,   308],
       [  243,   485]])

#Contoh Input

In [146]:
test_sample = x_test.sample(n=5)
test_sample

,final_text
4973,aplikasi otak akun saldo rp login gara ² kode ...
12528,apapun tu tf temen dgn nominal tau taunya uang...
123681,memuaskan harga transaksinya memuaskan hati ti...
25666,topup game dana bagus topup toko yg pembayaran...
97188,mohon perbaiki lahmasa gabisa transaksi gabisa...


In [147]:
test_sample_tfidf = tfidf.transform(test_sample)
test_sample_reshaped = np.reshape(test_sample_tfidf.toarray(), (test_sample_tfidf.shape[0], 1, test_sample_tfidf.shape[1]))

In [148]:
predicted_values = model.predict(test_sample_reshaped)
predicted_classes = np.argmax(predicted_values, axis=1)
sentiment = encoder.inverse_transform(predicted_classes)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


In [149]:
pd.DataFrame({'text': test_sample, 'sentiment': sentiment})

,text,sentiment
4973,aplikasi otak akun saldo rp login gara ² kode ...,negative
12528,apapun tu tf temen dgn nominal tau taunya uang...,negative
123681,memuaskan harga transaksinya memuaskan hati ti...,negative
25666,topup game dana bagus topup toko yg pembayaran...,negative
97188,mohon perbaiki lahmasa gabisa transaksi gabisa...,negative
